- Retrieve datasets and trim
- Do naive search
- do TFIDF search
- compare the 2 query searches
- make LDA model
- compare what LDA can tell us/ are the different summons similar to their respective types of summoning? (Specials and Normal)
- does a shift in rhetorical style affect topic inference? ? ?

- gg ez

In [ ]:
import math
import numpy as np
import pyGM as gm
import matplotlib.pyplot as plt
from collections import defaultdict


## open the Yu-Gi-Oh! corpus and read each line
file = open("card_text.txt", "r")
card_data = file.readlines()

YGOcorpus = list()
YGOword_set = list()
YGOword_count = defaultdict(int)

# loop through each dict in the file, add to YGOcorpus list and keep track of word occurences
for card in card_data:
    card_dict = eval(card)
    YGOcorpus.append(card_dict['Text'])
    YGOword_set += (card_dict['Text'])
    for word in card_dict['Text']:
        YGOword_count[word] +=1
    
YGOcorpus_size = len(YGOcorpus)

# Removes duplicates and rare words in YGOword_set
YGOvocab = set(YGOword_set)
for x in YGOword_count.keys():
    if(YGOword_count[x] <10):
        YGOvocab.remove(x)

YGOword_set_size = len(YGOvocab)

print(YGOcorpus_size)
print(YGOword_set_size)

# entire wordset count
#print(sorted(YGOword_count.items(), key = lambda x: x[1], reverse = True))

# list of every card's text and word counts
YGOwordCards = [dict.fromkeys(YGOword_set,0) for x in range(YGOcorpus_size)]

for i in range(YGOcorpus_size):
    for word in YGOcorpus[i]:
        YGOwordCards[i][word] +=1


        
# calculate TF, takes in a document's word frequency and the words in the document
def TF(wordDocs, text_list):
    tf = dict()
    
    text_list_size = len(text_list)
    
    for word, count in wordDocs.items():
        tf[word] = count/float(text_list_size)
        
    return tf


# idf is how significant a word is; the more a word appears, the less relevant and the lower the idf score
# takes list of documents and their word counts
def IDF(documents):
    # initializes to count
    idf = dict.fromkeys(documents[0].keys(),0)

    size = len(documents)

    for document in documents:
        for word, count in document.items():
            if count > 0:
                idf[word] += 1

    for word, count in idf.items():
        idf[word] = math.log((size/float(count)))

    return idf
# gets TFIDF of a document but multiplying its TF and IDF dicts
def TFIDF(tfs, idfs):
    
    TFIDFs = dict()
    
    for word, count in tfs.items():
        TFIDFs[word] = count * idfs[word]
        
    return TFIDFs


YGOtf_list = [(x, TF(YGOwordCards[x], YGOcorpus[x])) for x in range(YGOcorpus_size)]
YGOidfs = IDF(YGOwordCards) 

YGOtfidfs_list = [TFIDF(YGOtf_list[x][1],YGOidfs) for x in range(YGOcorpus_size)]



10082
885


In [7]:
## Naive search

search_1 = sorted(YGOtf_list, key = lambda x: x["summon"], reverse = True)
search_2 = sorted(YGOtf_list, key = lambda x: x["hand"], reverse = True)
search_3 = sorted(YGOtf_list, key = lambda x: x["graveyard"], reverse = True)
for x in range(1000):
    print()
    print(search_1[x]["summon"])
    


0.25
0.25
0.25
0.2222222222222222
0.2222222222222222
0.21052631578947367
0.2
0.2
0.2
0.2
0.2
0.2
0.18181818181818182
0.18181818181818182
0.18181818181818182
0.18181818181818182
0.17391304347826086
0.16666666666666666
0.16666666666666666
0.16666666666666666
0.16666666666666666
0.16666666666666666
0.16666666666666666
0.16666666666666666
0.16666666666666666
0.16666666666666666
0.16666666666666666
0.16666666666666666
0.16666666666666666
0.16666666666666666
0.16666666666666666
0.16666666666666666
0.16666666666666666
0.16666666666666666
0.16666666666666666
0.16666666666666666
0.16666666666666666
0.16666666666666666
0.16666666666666666
0.15789473684210525
0.15384615384615385
0.15384615384615385
0.15384615384615385
0.15384615384615385
0.15384615384615385
0.15384615384615385
0.15384615384615385
0.15384615384615385
0.15384615384615385
0.15384615384615385
0.15151515151515152
0.15
0.14285714285714285
0.14285714285714285
0.14285714285714285
0.14285714285714285
0.14285714285714285
0.1428571428571428

In [16]:
## LDA CODE

# Yu-Gi-Oh has 3 types of cards: spells, traps and monsters. We'll test using those 3 as topics.
#vocab_list = ["special", "summon", "fusion", "pendulum", "ritual", "synchro", "xyz",
#            "link", "normal", "flip", "set", "tribute", "target", "material", "hand",
#             "graveyard", "field", "monsters", "banished", "banish", "monster", "target",
#              "send", "negate", "add", "draw", "discard", "destroy", "remove", "inflict", "damage", "attack", "defense",
#              "extra", "deck", "equip", "activate", "gains", "attack", "position", "control", "detach", "scale", "co-linked",
#             "type", "tuner", "non-tuner", "level", "rating",
#             "polymerization", "rank", "points"]

vocab= {w:i for i,w in enumerate(YGOvocab)}
words = {i:w for i,w in enumerate(YGOvocab)}


w = np.array([vocab[x] for i,txt in enumerate(YGOcorpus) for x in txt if x in vocab], dtype = int)
d = np.array([i for i,txt in enumerate(YGOcorpus)])
D = max(d) + 1
W = max(w) + 1
N = len(d)

T = 3
beta = .001
alpha = .001

a = np.zeros((D,T), dtype = int)
b = np.zeros((W,T), dtype = int)
c = np.zeros((T,), dtype = int)

z = np.random.randint(T,size = N)

for n in range(N):
    a[d[n], z[n]] += 1
    b[w[n], z[n]] += 1
    c[z[n]] +=1
    

def gibbs(maxIter):
    for it in range(maxIter):
        for i in range(N):
            t = z[i]
            a[d[i],t] -=1
            b[w[i],t] -=1
            c[t] -=1
            
            probs = (beta + b[w[i],:])/(c[:] + beta*W) * (alpha + a[d[i],:])
            
            cumprobs = np.cumsum(probs)
            cumprobs /= cumprobs[-1]
            t = np.where(cumprobs > np.random.rand())[0][0]
            
            z[i] = t
            a[d[i], t] += 1
            b[w[i], t] += 1
            c[t] +=1 
            
def print_topics():
    
    for t in range(T):
        isort = np.argsort(-b[:,t])
        xsort = b[isort, t]
        print('[{}] ({:.3f}) {}'.format(t,c[t]/N,list(words[isort[ww]]for ww in range(10))))

print("No Iterations:\n")
print_topics()
print()
gibbs(50)        
print("After 50 Iterations:\n")
print()
print_topics()
print()
gibbs(100)
print("After 100 Iterations:\n")
print()
print_topics()
print()
gibbs(200)
print("After 200 Iterations:\n")
print()
print_topics()
print()

No Iterations:

[0] (0.333) ['card', 'monster', 'monsters', 'opponent', 'graveyard', 'special', 'target', 'effect', 'summon', 'field']
[1] (0.332) ['card', 'monster', 'opponent', 'special', 'field', 'graveyard', 'hand', 'target', 'effect', 'summon']
[2] (0.335) ['card', 'monster', 'special', 'effect', 'opponent', 'graveyard', 'summon', 'monsters', 'target', 'hand']

After 50 Iterations:


[0] (0.330) ['opponent', 'summon', 'effect', 'monster', 'control', 'face', 'type', 'summoned', 'deck', 'end']
[1] (0.332) ['card', 'monster', 'special', 'monsters', 'atk', 'destroy', 'destroyed', 'attack', 'trap', 'position']
[2] (0.338) ['monster', 'card', 'field', 'target', 'battle', 'graveyard', 'level', 'hand', 'phase', 'normal']

After 100 Iterations:


[0] (0.335) ['opponent', 'effect', 'monster', 'summon', 'hand', 'control', 'field', 'face', 'deck', 'type']
[1] (0.333) ['card', 'special', 'monsters', 'atk', 'destroy', 'destroyed', 'attack', 'position', 'trap', 'monster']
[2] (0.332) ['monster',

In [19]:
for i in range(len(a)):print(str(a[i])+": "+" ".join(YGOcorpus[i]))

[0 0 1]: target monster 1000 less atk graveyard special summon attack position target changed defense position destroy card card leaves field destroy target target destroyed destroy card
[1 0 0]: zombie suddenly appeared plot 13 empty grave
[0 1 0]: pendulum effect main phase add face pendulum monster extra deck hand effect savant thomas monster effect target card pendulum zone destroy card special summon level 8 monster deck defense position rest effects any negated any battle damage opponent takes becomes halved effect savant thomas
[0 1 0]: tribute gem knight monster target gem knight monster graveyard special summon target graveyard
[0 1 0]: spellcaster type monster declares attack player special summons spellcaster type monster 2000 less atk deck attack position
[0 0 1]: flip zombie type monsters gain 200 atk def next 4 standby phases zombie type monsters gains 200 atk def effects last long card face field
[0 1 0]: cipher xyz monster control xyz material destroyed battle opponent 

[1 0 0]: target majespecter monster control end atk def become double current atk def any battle damage inflicts opponent halved
[0 1 0]: discard card hand destroy face continuous trap cards
[1 0 0]: tribute ninja monster target monster opponent controls banish target card banished monster card zone used face card leaves field return card banished effect same monster card zone same battle position
[0 1 0]: earth machine type monster normal special summoned field except damage step special summon card hand original atk def become halved effect heavy freight train derricrane card detached xyz monster sent graveyard activate monster effect target card opponent controls destroy
[0 0 1]: target face cards field number attack position abyss actor monsters different names control destroy control level 7 higher abyss actor monster opponent activate cards effects response card activation set card owner control destroyed opponent card effect have face abyss actor pendulum monster extra deck add 

[1 0 0]: 2 number xyz monsters same rank have xyz material special summon rank 9 lower number monsters 3000 less atk have different ranks extra deck number xyz materials different names attached card effects negated detach xyz material card rest unless effect negated special summon any battle damage opponent takes becomes halved control another number xyz monster card destroyed battle card effects
[0 1 0]: 2 masked hero monsters card always treated elemental hero card must fusion summoned special summoned ways face field card light attribute player target face card field negate target effects end
[0 1 0]: destroyed battle resolving card effect targets face card destroy card card inflicts battle damage opponent direct attack discard random card hand
[0 1 0]: remove bushido counter side field activate effects target monster opponent controls destroy target target card opponent controls return target hand
[0 1 0]: card unaffected monsters activated effects except pendulum monsters
[0 0 1]

[0 1 0]: monster normal summoned side field except master flaming dragonswords increase monster level card gains 300 atk end phase
[1 0 0]: discard monster card hand graveyard select monster side field lower level discarded monster selected monster attack twice
[1 0 0]: card sent hand deck graveyard opponent card effect special summon card
[0 1 0]: pay 1000 life points special summon level 6 lower fusion monster extra deck face attack defense position fusion monster attack opponent life points directly returned extra deck end
[0 1 0]: special summon card graveyard returning dark world monster control hand except grapha dragon lord dark world card discarded graveyard card effect target card opponent controls destroy target card discarded hand graveyard opponent card effect look random card opponent hand monster special summon side field
[0 1 0]: player reveals card hand one card higher level card lower level sent graveyard player revealed takes 1000 damage non monster cards treated leve

[0 0 1]: fast lethal creature very dangerous claws
[1 0 0]: target set card opponent spell trap zone card face field target activated
[0 1 0]: tuner non tuner monsters equip card equipped card inflict 800 damage opponent target equip card equipped card send graveyard equip card equipped card sent graveyard except damage step target face monster opponent controls destroy target
[0 1 0]: uria lord searing flames hamon lord striking thunder raviel lord phantasms must first special summoned extra deck banishing above cards control polymerization destroyed battle card gains 10 000 atk
[1 0 0]: special summoned end phase card normal summoned flipped face return hand card normal summoned flipped face reveal spirit monster hand except rasetsu target face attack position monster opponent controls return target hand special summon activate effect
[0 1 0]: raidraptor xyz monster destroyed sent graveyard controlled target raidraptor xyz monster graveyard special summon special summon extra deck ra

...

[0 1 0]: flip add equip spell card deck hand
[0 0 1]: 2 level 8 dark monsters must xyz summoned special summoned ways player detach xyz material card send card hand graveyard target face card opponent controls change card defense position face card effects negated end
[0 0 1]: golem massive right hand crushing victims
[1 0 0]: genex tuner non tuner wind monsters opponent select card attack target card destroys opponent monster battle add genex monster deck hand
[0 0 1]: destroy many face spell trap cards field possible card player takes damage equal total number cards destroyed effect x 300
[0 0 1]: card normal summoned opponent controls special summoned monster target blackwing tuner graveyard special summon monsters extra deck rest except blackwing monsters special summon monster effects negated
[0 1 0]: summoned unless control face wicked breaking flamberge baou card destroys opponent monster battle sends graveyard gains 1000 atk negate effects monsters destroyed battle card
[1 0 0]

[0 1 0]: equipped monster attack atk def become 100 standby phases inflict 500 damage controller equipped monster
[0 0 1]: warrior type monsters control gain 400 atk
[1 0 0]: card special summoned battle damage player battle involving monster becomes 0 card destroyed battle distribute new fog counters equal level monster destroyed among face monsters
[1 0 0]: opponent controls monster control monsters special summon card hand attack summoned control toon world opponent controls toon monsters card attack opponent directly
[0 1 0]: token special summoned destroy many tokens field possible inflict 300 damage opponent token destroyed
[1 0 0]: monster appears midnight tribute made
[1 0 0]: target level 5 higher dinosaur type monster control dinosaur type monster still face field choose 2 monsters opponent controls have lower level face down return hand
[0 1 0]: battle position rock type monster control changed target spell trap card opponent controls destroy target
[0 1 0]: end phase card s

[0 0 1]: pendulum effect attack declared involving face monster control opponent monster have odd eyes card pendulum zone make monster control gain 1200 atk end battle phase even card leaves field monster effect pendulum summoned card inflicts battle damage opponent attacking inflict damage opponent equal number odd eyes cards pendulum zones x 1200 effect odd eyes phantom dragon
[0 1 0]: card inflicts battle damage opponent send card top opponent deck graveyard face light monster control
[0 1 0]: 2 tuners non tuner vylon monster card synchro summoned destroy face monsters normal summoned set target vylon monster graveyard equip target card player monster effect activates send equip card equipped card graveyard negate activation destroy monster
[0 0 1]: special summon card hand banishing spell card graveyard card attack directly take battle damage attacks involving card
[1 0 0]: target mecha phantom beast monster field atk becomes double original atk unaffected spell trap effects attack

[1 0 0]: 2 monsters face field quick effect target monsters field gy number monsters linked card return hand monster card points destroyed battle sent gy special summon monster hand
[0 0 1]: 2 gouki monsters monsters field lose atk equal original def card destroyed battle card effect destroy monsters card points instead
[0 0 1]: 2 cyberse monsters end damage step monster card points attacked opponent link monster activate effect monster attacked gains additional attack opponent link monster battle phase link monster currently control except card monsters except monster attack activate effect
[1 0 0]: level 4 lower cyberse monster tribute monster card points draw card place card hand bottom deck effect link disciple
[0 0 1]: 2 rokket monsters target link monster control special summon rokket monster hand zone link monster points used link material destroy end phase effect reloader dragon card destroyed battle sent gy target rokket monster gy add hand
[0 1 0]: 2 effect monsters another m

[1 0 0]: target zombie monster field end gains 1000 atk loses 1000 atk main phase card gy shuffle banished zombie monsters deck set card banish leaves field effect zombie power struggle
[0 0 1]: face fire monster control destroyed battle opponent card effect special summon card hand inflict damage opponent equal half atk destroyed fire monsters gy effect cataclysmic scorching sunburner
[1 0 0]: card treated 2 tributes tribute summon following effects world legacy world ark link monster control destroyed opponent card effect sent gy send card hand gy target link monsters special summon opponent special summons monster extra deck control normal summoned set card send monster deck gy
[0 1 0]: card possession destroyed opponent card sent gy shuffle card deck following effects madolche petingcessoeur have monsters gy special summon card hand card special summoned special summon madolche monster hand deck except madolche petingcessoeur reduce level special summon monsters rest except madolch

In [21]:
for j in range(len(b)):print(str(b[j])+": "+words[j])

[0 0 0]: quant
[ 0  0 75]: spell
[ 0 34  0]: points
[0 2 0]: two
[0 0 0]: artifact
[0 0 0]: like
[ 0  0 37]: draw
[1 0 0]: flamvell
[0 0 4]: die
[0 1 0]: man
[0 0 1]: pick
[0 0 0]: salamangreat
[2 0 0]: rolls
[ 0  0 11]: using
[0 0 1]: spellbook
[0 3 0]: performage
[0 0 0]: gemini
[0 0 0]: emperor
[1 0 0]: randomly
[52  0  0]: add
[0 0 1]: scrap
[2 0 0]: frog
[0 4 0]: flower
[0 3 0]: twice
[0 1 0]: mirror
[0 1 0]: again
[2 0 0]: increased
[0 5 0]: dinomist
[0 0 0]: chalice
[29  0  0]: activated
[0 1 0]: wizard
[6 0 0]: abyss
[0 1 0]: flames
[0 0 0]: multiple
[0 0 0]: drew
[ 0  0 14]: materials
[0 0 0]: dwells
[0 0 2]: core
[0 0 3]: possession
[ 0 17  0]: have
[2 0 0]: dinosaur
[0 0 6]: resolves
[0 0 3]: one
[0 0 0]: valkyrie
[4 0 0]: insect
[ 0 17  0]: instead
[ 0 12  0]: token
[ 0  0 10]: inflicts
[ 0  0 46]: gains
[0 0 0]: thief
[7 0 0]: flipped
[0 2 0]: souls
[0 0 2]: soul
[  0   0 124]: battle
[0 0 0]: win
[0 0 0]: terms
[0 0 1]: last
[0 1 0]: link
[56  0  0]: player
[0 2 0]: genex

In [20]:
print(YGOword_count["detach"])

430
